In [8]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

In [2]:
config = {'df_path': '../input/user1.features_labels.csv'}
har = utils.HAR(config)
har.dummy_test()

2022-03-11 17:19:08.444815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-11 17:19:08.506214: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-11 17:19:08.506231: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-11 17:19:08.506679: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Epoch 1/40
161/161 [==============================] - 1s 2ms/step - loss: 0.4620 - categorical_accuracy: 0.6238 - val_loss: 0.3765 - val_categorical_accuracy: 0.7413
Epoch 2/40
161/161 [==============================] - 0s 886us/step - loss: 0.3519 - categorical_accuracy: 0.7786 - val_loss: 0.3234 - val_categorical_accuracy: 0.8035
Epoch 3/40
161/161 [==============================] - 0s 844us/step - loss: 0.3177 - categorical_accuracy: 0.8029 - val_loss: 0.2985 - val_categorical_accuracy: 0.8284
Epoch 4/40
161/161 [==============================] - 0s 1ms/step - loss: 0.3001 - categorical_accuracy: 0.8184 - val_loss: 0.2932 - val_categorical_accuracy: 0.8358
Epoch 5/40
161/161 [==============================] - 0s 1ms/step - loss: 0.2910 - categorical_accuracy: 0.8259 - val_loss: 0.2794 - val_categorical_accuracy: 0.8383
Epoch 6/40
161/161 [==============================] - 0s 867us/step - loss: 0.2829 - categorical_accuracy: 0.8246 - val_loss: 0.2753 - val_categorical_accuracy: 0.843

In [3]:
data = pd.read_csv('../input/user1.features_labels.csv')#.dropna() #TODO: attention

#data.isna().sum()
data.drop(columns=['lf_measurements:temperature_ambient'], inplace=True)
data[data.columns.drop(data.filter(regex='label:'))]

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,lf_measurements:screen_brightness,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source
0,1464129912,1.011438,0.012573,0.023013,0.041240,1.006184,1.010868,1.016028,0.935099,6.684536,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
1,1464129950,1.011233,0.009356,-0.005622,0.016687,1.006338,1.010926,1.016657,1.732968,6.684569,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
2,1464130031,1.013422,0.018068,-0.008593,0.039286,1.004077,1.012983,1.021926,1.464639,6.684453,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
3,1464130109,1.014891,0.016400,0.021383,0.038825,1.005934,1.014670,1.023000,1.440043,6.684483,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
4,1464130130,1.017487,0.022632,-0.012891,0.037226,1.006040,1.017587,1.028168,1.937362,6.684364,...,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1464886422,1.002114,0.043888,-0.023421,0.060480,0.975662,1.001421,1.028555,2.562570,6.683649,...,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2681,1464886722,1.015957,0.068057,-0.030021,0.108892,0.980304,1.015067,1.050255,2.055226,6.682352,...,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2682,1464887023,0.996011,0.039554,-0.034651,0.065742,0.976391,0.998637,1.018739,1.989673,6.683814,...,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2683,1464887322,1.079283,0.404817,0.330815,0.569329,0.907679,1.001940,1.298900,2.453154,6.612619,...,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4


In [4]:
data[data.columns.drop(data.filter(regex='label:'))].dropna()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,lf_measurements:screen_brightness,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source


In [5]:
data.fillna(0.0, inplace=True)
#data.info()
data[data.columns.drop(data.filter(regex='label:'))].dropna()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,lf_measurements:screen_brightness,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source
0,1464129912,1.011438,0.012573,0.023013,0.041240,1.006184,1.010868,1.016028,0.935099,6.684536,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
1,1464129950,1.011233,0.009356,-0.005622,0.016687,1.006338,1.010926,1.016657,1.732968,6.684569,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
2,1464130031,1.013422,0.018068,-0.008593,0.039286,1.004077,1.012983,1.021926,1.464639,6.684453,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
3,1464130109,1.014891,0.016400,0.021383,0.038825,1.005934,1.014670,1.023000,1.440043,6.684483,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
4,1464130130,1.017487,0.022632,-0.012891,0.037226,1.006040,1.017587,1.028168,1.937362,6.684364,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,1464886422,1.002114,0.043888,-0.023421,0.060480,0.975662,1.001421,1.028555,2.562570,6.683649,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2681,1464886722,1.015957,0.068057,-0.030021,0.108892,0.980304,1.015067,1.050255,2.055226,6.682352,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2682,1464887023,0.996011,0.039554,-0.034651,0.065742,0.976391,0.998637,1.018739,1.989673,6.683814,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
2683,1464887322,1.079283,0.404817,0.330815,0.569329,0.907679,1.001940,1.298900,2.453154,6.612619,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4


In [6]:
har.mlp.model.save('model/saved_model')

2022-03-11 17:23:04.276739: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/saved_model/assets


In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model('model/saved_model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model/model.tflite', 'wb') as f:
  f.write(tflite_model)

2022-03-11 17:25:06.806404: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-11 17:25:06.806426: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-11 17:25:06.806430: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-11 17:25:06.807108: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model
2022-03-11 17:25:06.808538: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-11 17:25:06.808552: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model
2022-03-11 17:25:06.811993: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-11 17:25:06.837873: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: model/saved_model